In [ ]:

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

%matplotlib inline
df = pd.read_csv(r'train_clean2.csv', error_bad_lines=False,encoding="cp949")
data = df[['abstract']]
data['abstract'] = data.apply(lambda row: nltk.word_tokenize(row['abstract']), axis=1) #문제해결을 위한 토큰화


from nltk.stem import WordNetLemmatizer
data['abstract'] = data['abstract'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

tokenized_doc = data['abstract'].apply(lambda x: [word for word in x if len(word) > 3])



# TODO 토큰화 되돌리기
detokenized_doc = []
for i in range(len(data)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

data['abstract'] = detokenized_doc


from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english') # 상위 1,000개의 단어를 보존
#vectorizer = TfidfVectorizer(stop_words='english') 아마 max_features없이 그냥 stop_words만 매개변수로 넣어줘도 될 듯
#a = vectorizer.fit_transform(text['abstract']).toarray()
#print(a.shape) 총 단어 개수 확인


data_vectorized = vectorizer.fit_transform(data['abstract'])

from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=15,learning_method='online',random_state=777,max_iter=1)


lda_top=lda_model.fit_transform(data_vectorized)

lda_output = lda_model.transform(data_vectorized)
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
docnames = ["Doc" + str(i) for i in range(len(data))]
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics

